<a href="https://colab.research.google.com/github/pearl-yu/twitch_project/blob/main/data_collection_5000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dependent variable construction: 
Streaming time. 

In [2]:
## Run this line if using Colab.
!git clone https://github.com/pearl-yu/twitch_project
%cd twitch_project/

Cloning into 'twitch_project'...
remote: Enumerating objects: 294, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 294 (delta 107), reused 130 (delta 79), pack-reused 122
Receiving objects: 100% (294/294), 8.69 MiB | 6.87 MiB/s, done.
Resolving deltas: 100% (166/166), done.
/content/twitch_project


In [1]:
## So we can write the dataset to google drive (not github. need to upload to github manually for now.)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/twitch_data

/content/drive/MyDrive/twitch_data


In [3]:
%ls

1013713136.mp4                  raid_df.csv       sentiments5
chat_df.csv                     sample_streamer/  vid_df_wraid.csv
chat_features_womeaningful.csv  sentiments        video_df_merge2.csv
data_vid_df_l.csv               sentiments1       video_df_merge3.csv
features_vid_df_lag.csv         sentiments2       video_df_merge.csv
features_vid_df_lag_raid.csv    sentiments3
raid/                           sentiments4


In [4]:
import requests  # get the content from the API by using the get() method. json() method converts the API response to JSON format for easy handling.
import json   # to work with the returned json content from API.
import numpy as np
import pandas as pd

from pandas import json_normalize

import matplotlib.pyplot as plt

from datetime import datetime
import re

In [8]:
## Pearl's credentials
client_id = 'tamrylbvlu3wub4r8c5fddwsou246x'
client_secret= 'y4enh38kpiwkh7j00nqvv185ttsedw'

# Defining the URL to which we will make the request
url = 'https://id.twitch.tv/oauth2/token'
# Defining the parameters to be passed to the request
body = {
  'client_id'     : client_id,
  'client_secret' : client_secret,
  'grant_type'    : 'client_credentials'
}

access_code = requests.post(url, params=body) # Making a POST request to the URL to retrieve the access token
access_token = json.loads(access_code.text) #access token response is a JSON-encoded app access token

access_token = access_token['access_token']

With the sample:
- Pull user ids by calling the https://dev.twitch.tv/docs/api/reference#get-users endpoint, specifying the login parameter.
- Pull the video ids following this: https://github.com/btjmga/twitch-getallvods  (I didn't have the code to pull ALL the vids)
- Pull the chatfiles of these videos. (using the downloader https://github.com/lay295/TwitchDownloader is easy. Check  https://github.com/Chaparro/rechat-dl and https://github.com/malciin/twitch-comments-downloader for code.)
- Construct variables from the chatfile.
- Call what other stuff? 

In [9]:
# reference - read in multiple files: https://sparkbyexamples.com/pandas/pandas-read-multiple-csv-files/
# Import libraries
import glob

# Get CSV files list from a folder
path = './sample_streamer' # only English channels. Top 5000 followed.
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
df_list = (pd.read_csv(file) for file in csv_files) # This creates a list of dataframes
df = pd.concat(df_list) # Concatenate all DataFrames

df = df.sample(frac=1).reset_index() # shuffle the order of rows
print(df.shape) 
df.head()

#df_en = df[df['Language'] == 'English'].reset_index()

(5100, 14)


,index,Unnamed: 0,Unnamed: 1,Channel,Watch time (mins),Stream time (mins),Peak viewers,Average viewers,Followers,Followers gained,Partnered,Mature,Language,Unnamed: 12
0,59,1060,https://static-cdn.jtvnw.net/jtv_user_pictures...,fruitberries,883245,1695,6137,521,231339,1046,True,False,English,fruitberries
1,86,87,https://static-cdn.jtvnw.net/jtv_user_pictures...,Warframe,25722690,1275,45495,20174,1783801,8121,True,False,English,Warframe
2,98,399,https://static-cdn.jtvnw.net/jtv_user_pictures...,AnthonyZ,41879250,14055,8062,2979,585440,-673,True,False,English,AnthonyZ
3,99,3700,https://static-cdn.jtvnw.net/jtv_user_pictures...,KatieLouise,430260,4065,254,105,58397,281,True,True,English,KatieLouise
4,26,427,https://static-cdn.jtvnw.net/jtv_user_pictures...,kkatamina,6925950,1980,11910,3497,544927,5010,True,False,English,kkatamina


In [ ]:
def api_info(url = 'https://api.twitch.tv/helix/', key = 'user_id',key_series = df['Channel'], n = 5):
    '''
    url is the api URL
    key_series is a pandas series of the keys. check on the API reference page to see what should be the key
    n is the number of keys you wanna pull info from
    '''
    headers = {
            'Authorization' : 'Bearer '+str(access_token),
            'Client-Id' : client_id
        }
    keys_list = list(key_series) 

    temp_df = pd.DataFrame()
    for i in range(0,n) :
        payload = {key:keys_list[100*i:100*(i+1)]}
        temp_response = requests.get(url, params = payload, headers=headers)
        # Load the JSON
        temp_response_json = json.loads(temp_response.text)
        if list(temp_response_json.keys())[0] == 'error':
            print('error')
            continue
        else:
            temp_data = temp_response_json['data']
            temp_df_temp = pd.DataFrame.from_dict(json_normalize(temp_data), orient='columns')
        
            frames = [temp_df, temp_df_temp]
            temp_df = pd.concat(frames, ignore_index=True)
    
    print(temp_df.shape)
    print(temp_df.columns)
    
    return temp_df

In [ ]:
## call the get_users end points to retrieve user id
user_df = api_info(url = 'https://api.twitch.tv/helix/users',
                   key='login',
                   key_series = df['Channel'], 
                   n = 60)

In [ ]:
print(user_df.shape)
user_df.head()

(4186, 10)


,id,login,display_name,type,broadcaster_type,description,profile_image_url,offline_image_url,view_count,created_at
0,39513373,pixelbypixelstudios,PixelbyPixelStudios,,partner,Game Developers of Marbles on Stream,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,924238,2013-01-20T11:45:36Z
1,31500812,smitegame,SmiteGame,,partner,"God-like action, all the time. This is the off...",https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,204005335,2012-06-20T21:05:55Z
2,435653871,chels,Chels,,partner,"Whatever you are, be a good one. 🧡",https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,340509,2019-05-14T13:03:34Z
3,523497767,kyreewalker1,kyreewalker1,,affiliate,THE OFFICIAL CHANNEL OF KYREE WALKER FOLLOW ME...,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,149956,2020-05-01T02:45:04Z
4,8221220,jaieazy,JaiEazy,,partner,BEEN HERE SINCE THE BEGINNING PLAYING COD AND ...,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,5795254,2009-09-11T20:17:02Z


In [ ]:
def api_info(url = 'https://api.twitch.tv/helix/users?id=', key_series = df['Channel'], n = 5):
    '''
    url is the api URL
    key_series is a pandas series of the keys. check on the API reference page to see what should be the key
    n is the number of keys you wanna pull info from
    '''
    headers = {
            'Authorization' : 'Bearer '+str(access_token),
            'Client-Id' : client_id
        }

    temp_df = pd.DataFrame()
    for i in range(0,n) :
        temp_response = requests.get(url+str(key_series[i]), headers=headers)
        # Load the JSON
        temp_response_json = json.loads(temp_response.text)
        if list(temp_response_json.keys())[0] == 'error':
            continue
        else:
            temp_data = temp_response_json['data']
            temp_df_temp = pd.DataFrame.from_dict(json_normalize(temp_data), orient='columns')
        
            frames = [temp_df, temp_df_temp]
            temp_df = pd.concat(frames, ignore_index=True)
    
    print(temp_df.shape)
    print(temp_df.columns)
    
    return temp_df

In [ ]:
## call the get_videos end points to retrieve all video ids of the users
video_df = api_info(url = 'https://api.twitch.tv/helix/videos?user_id=',
         key_series = user_df['id'], 
         n = len(user_df))

(69639, 17)
Index(['id', 'stream_id', 'user_id', 'user_login', 'user_name', 'title',
       'description', 'created_at', 'published_at', 'url', 'thumbnail_url',
       'viewable', 'view_count', 'language', 'type', 'duration',
       'muted_segments'],
      dtype='object')


In [ ]:
print(video_df.shape)
print(video_df.user_id.nunique())
video_df.head()

(69639, 17)
3950


,id,stream_id,user_id,user_login,user_name,title,description,created_at,published_at,url,thumbnail_url,viewable,view_count,language,type,duration,muted_segments
0,1663226548,1,39513373,pixelbypixelstudios,PixelbyPixelStudios,[01 Oct. '22] Devvy (Plinko & Royale),,2022-11-26T15:05:39Z,2022-11-26T15:05:39Z,https://www.twitch.tv/videos/1663226548,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9...,public,1,en,highlight,6h5m9s,None
1,1663226212,1,39513373,pixelbypixelstudios,PixelbyPixelStudios,"[30 Sep. '22] Devvy (Plinko, Royale, Skins, UI)",,2022-11-26T15:05:08Z,2022-11-26T15:05:08Z,https://www.twitch.tv/videos/1663226212,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9...,public,0,en,highlight,8h20m54s,None
2,1663206760,1,39513373,pixelbypixelstudios,PixelbyPixelStudios,[29 Sep. '22] The Byte #018 Uncut,,2022-11-26T14:32:56Z,2022-11-26T14:32:56Z,https://www.twitch.tv/videos/1663206760,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9...,public,0,en,highlight,59m21s,None
3,1663191849,1,39513373,pixelbypixelstudios,PixelbyPixelStudios,[28 Sep. '22] Devvy (Plinko & Sprint),,2022-11-26T14:06:02Z,2022-11-26T14:06:02Z,https://www.twitch.tv/videos/1663191849,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9...,public,0,en,highlight,3h33m50s,None
4,1663191728,1,39513373,pixelbypixelstudios,PixelbyPixelStudios,[26 Sep. '22] Devvy (Plinko) + Bingo,,2022-11-26T14:05:50Z,2022-11-26T14:05:50Z,https://www.twitch.tv/videos/1663191728,https://static-cdn.jtvnw.net/cf_vods/d1m7jfoe9...,public,0,en,highlight,3h36m23s,None


In [ ]:
# Some pre-processing of the video_df
video_df['created_at'] = pd.to_datetime(video_df['created_at']) 
video_df['duration_min'] = video_df['duration'].str.replace("h", '*60+').str.replace('m','*1+').str.replace('s','*(1/60)').apply(eval) # convert duration to numeric in minutes

In [ ]:
# Get some simple statistics
print(video_df['created_at'].min(),video_df['created_at'].max())

video_df.groupby('user_id').agg({'id':'count',
                                 'created_at': [min,max],
                                 'duration_min':[min,max,'mean'],
                                 'view_count':'mean'})

2009-01-17 10:59:38+00:00 2022-11-27 17:01:55+00:00


id                created_at                            \
          count                       min                       max   
user_id                                                               
10004253     20 2022-10-16 04:21:49+00:00 2022-11-24 07:13:08+00:00   
100182904    20 2022-11-16 08:25:22+00:00 2022-11-27 14:53:22+00:00   
10027602     20 2022-11-20 02:53:59+00:00 2022-11-25 21:23:09+00:00   
100356138    20 2022-11-01 23:23:24+00:00 2022-11-23 03:45:36+00:00   
100372176    20 2022-10-30 16:14:03+00:00 2022-11-27 14:47:27+00:00   
...         ...                       ...                       ...   
99675008      9 2020-12-27 16:41:59+00:00 2022-11-16 19:17:51+00:00   
99710196     10 2022-08-10 02:33:21+00:00 2022-11-27 02:46:26+00:00   
99723741     20 2021-01-29 22:15:05+00:00 2022-11-13 03:07:03+00:00   
99801146     20 2022-10-19 21:27:01+00:00 2022-11-24 22:20:36+00:00   
99816984      6 2020-01-15 05:47:19+00:00 2022-11-15 21:48:40+00:00   

          duration_min                         view_count  
                   min         max        mean       mean  
user_id                                                    
10004253      0.533333  434.900000  134.544167     400.65  
100182904     0.333333  958.000000  214.680000   18944.55  
10027602      3.183333  213.066667   57.894167    1417.85  
100356138     3.033333  622.866667  268.760000    2920.00  
100372176   131.383333  636.766667  361.168333    8992.15  
...                ...         ...         ...        ...  
99675008    120.333333  340.216667  236.822222   44315.00  
99710196     38.616667  353.800000  200.436667    2242.30  
99723741     15.366667  312.266667  210.670833    8819.55  
99801146     26.233333  288.033333  149.268333    4998.20  
99816984     73.766667  189.166667  137.516667   11056.50  

[3950 rows x 7 columns]

In [ ]:
user_temp = user_df[['id', 'broadcaster_type','description', 'profile_image_url', 'offline_image_url', 'created_at']]
user_temp.columns = ['user_id','broadcaster_type','broadcast_description','broadcaster_profile_image_url', 'broadcaster_offline_image_url', 'broadcaster_created_at']

video_df_merge= pd.merge(video_df, 
                         user_temp,  
                         how='left', left_on=['user_id'], right_on = ['user_id'])

print(video_df_merge.columns)
video_df_merge.head()

Index(['id', 'stream_id', 'user_id', 'user_login', 'user_name', 'title',
       'description', 'created_at', 'published_at', 'url', 'thumbnail_url',
       'viewable', 'view_count', 'language', 'type', 'duration',
       'muted_segments', 'duration_min', 'broadcaster_type',
       'broadcast_description', 'broadcaster_profile_image_url',
       'broadcaster_offline_image_url', 'broadcaster_created_at'],
      dtype='object')


,id,stream_id,user_id,user_login,user_name,title,description,created_at,published_at,url,...,language,type,duration,muted_segments,duration_min,broadcaster_type,broadcast_description,broadcaster_profile_image_url,broadcaster_offline_image_url,broadcaster_created_at
0,1663226548,1,39513373,pixelbypixelstudios,PixelbyPixelStudios,[01 Oct. '22] Devvy (Plinko & Royale),,2022-11-26 15:05:39+00:00,2022-11-26T15:05:39Z,https://www.twitch.tv/videos/1663226548,...,en,highlight,6h5m9s,None,365.150000,partner,Game Developers of Marbles on Stream,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-01-20T11:45:36Z
1,1663226212,1,39513373,pixelbypixelstudios,PixelbyPixelStudios,"[30 Sep. '22] Devvy (Plinko, Royale, Skins, UI)",,2022-11-26 15:05:08+00:00,2022-11-26T15:05:08Z,https://www.twitch.tv/videos/1663226212,...,en,highlight,8h20m54s,None,500.900000,partner,Game Developers of Marbles on Stream,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-01-20T11:45:36Z
2,1663206760,1,39513373,pixelbypixelstudios,PixelbyPixelStudios,[29 Sep. '22] The Byte #018 Uncut,,2022-11-26 14:32:56+00:00,2022-11-26T14:32:56Z,https://www.twitch.tv/videos/1663206760,...,en,highlight,59m21s,None,59.350000,partner,Game Developers of Marbles on Stream,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-01-20T11:45:36Z
3,1663191849,1,39513373,pixelbypixelstudios,PixelbyPixelStudios,[28 Sep. '22] Devvy (Plinko & Sprint),,2022-11-26 14:06:02+00:00,2022-11-26T14:06:02Z,https://www.twitch.tv/videos/1663191849,...,en,highlight,3h33m50s,None,213.833333,partner,Game Developers of Marbles on Stream,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-01-20T11:45:36Z
4,1663191728,1,39513373,pixelbypixelstudios,PixelbyPixelStudios,[26 Sep. '22] Devvy (Plinko) + Bingo,,2022-11-26 14:05:50+00:00,2022-11-26T14:05:50Z,https://www.twitch.tv/videos/1663191728,...,en,highlight,3h36m23s,None,216.383333,partner,Game Developers of Marbles on Stream,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-01-20T11:45:36Z


In [ ]:
video_df_merge.to_csv('video_df_merge3.csv')

## Chats

In [5]:
## retrieve chat files of these videos.
# https://pypi.org/project/chat-downloader/
!pip install chat-downloader  # Need to install this everytime if you're using Colab.
from chat_downloader import ChatDownloader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 80 kB 2.6 MB/s 
     |████████████████████████████████| 41 kB 89 kB/s 
     |████████████████████████████████| 55 kB 2.4 MB/s 


In [6]:
video_df_merge = pd.read_csv('video_df_merge3.csv')
print(video_df_merge.shape)
print(video_df_merge.columns)
# video_df_merge.head()

(69819, 24)
Index(['Unnamed: 0', 'id', 'stream_id', 'user_id', 'user_login', 'user_name',
       'title', 'description', 'created_at', 'published_at', 'url',
       'thumbnail_url', 'viewable', 'view_count', 'language', 'type',
       'duration', 'muted_segments', 'duration_min', 'broadcaster_type',
       'broadcast_description', 'broadcaster_profile_image_url',
       'broadcaster_offline_image_url', 'broadcaster_created_at'],
      dtype='object')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
## First we take a look at what data there is about the chats
url = video_df_merge['url'][100]
chat = ChatDownloader().get_chat(url)       # create a generator
         # print the formatted message

# For each chat message, this is the full structure
next(chat)

{'message_id': 'FCgkbqUwRxYhmQ',
 'timestamp': 1605305277348656,
 'time_in_seconds': 0,
 'author': {'display_name': 'hannahwaslonely',
  'id': '605967091',
  'name': 'hannahwaslonely',
  'type': 'user',
  'bio': None,
  'created_at': 1605044185190250,
  'images': [{'url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/be33b2f5-b999-4f9c-befd-fdb3263535d2-profile_image-300x300.png',
    'width': 300,
    'height': 300,
    'id': '300x300'},
   {'url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/be33b2f5-b999-4f9c-befd-fdb3263535d2-profile_image-70x70.png',
    'width': 70,
    'height': 70,
    'id': '70x70'}]},
 'source': 'chat',
 'state': 'published',
 'message': 'Sks',
 'time_text': '0:00',
 'message_type': 'text_message'}

In [8]:
next(chat)

{'message_id': '3DwkbqUwRxZmvg',
 'timestamp': 1605305277375355,
 'time_in_seconds': 1,
 'author': {'display_name': 'pigpig123funnymic',
  'id': '590876231',
  'name': 'pigpig123funnymic',
  'type': 'user',
  'bio': None,
  'created_at': 1601579679405656,
  'images': [{'url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/7648880e-b7e2-48c9-b197-1190223be244-profile_image-300x300.png',
    'width': 300,
    'height': 300,
    'id': '300x300'},
   {'url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/7648880e-b7e2-48c9-b197-1190223be244-profile_image-70x70.png',
    'width': 70,
    'height': 70,
    'id': '70x70'}],
  'colour': '#00FF7F'},
 'source': 'chat',
 'state': 'published',
 'message': 'can you ban me',
 'time_text': '0:01',
 'message_type': 'text_message'}

In [9]:
next(chat).keys()

dict_keys(['message_id', 'timestamp', 'time_in_seconds', 'author', 'source', 'state', 'message', 'time_text', 'message_type'])

In [10]:
next(chat)['author'].keys()

dict_keys(['display_name', 'id', 'name', 'type', 'bio', 'created_at', 'images', 'badges'])

# **About the author, there're badges information that indicates if subscriber, if VIP, if partner streamer, etc. These should definitely be the controls. Also cheermotes and Emotes?**

In [11]:
# define function to get chat files.
import itertools

def get_chat(key_series = video_df_merge['url'],n = 5, n_message_limit = 1000):
  '''
    key_series is a pandas series of video urls.
    n is the number of videos you wanna pull chat from
    n_message_limit is the limit of messages you wanna pull from each video
    '''
  timestamp = []
  time_in_seconds = []
  messages = []
  author_id = []
  author_name = []
  author_bio = []
  #author_color = []
  video_url = []

  for i in range(0,n+1) :
    try:
      url = key_series[i]
      chat = ChatDownloader().get_chat(url)   
      top = itertools.islice(chat, n_message_limit)
      
      counter = len(messages)

      for message in top:     # iterate over messages
        video_url.append(url)                     
        timestamp.append(message['timestamp'])
        time_in_seconds.append(message['time_in_seconds'])
        author_id.append(message['author']['id'])
        author_name.append(message['author']['display_name']) 
        author_bio.append(message['author']['bio'])
        #author_color.append(message['author']['colour'])
        messages.append(message['message'])
      
      print(i / n, url,len(messages)-counter)
    except:
      print('error')
      pass

  chat_df = pd.DataFrame(list(zip(video_url,timestamp,time_in_seconds,author_id,author_name,author_bio,messages)),
                         columns = ['video_url','timestamp','time_in_seconds','author_id','author_name','author_bio','messages'])
  
  return chat_df

In [12]:
video_df_merge['duration_min'].describe()

count    69639.000000
mean       267.790152
std        270.650281
min          0.000000
25%        107.916667
50%        226.500000
75%        361.900000
max       2880.066667
Name: duration_min, dtype: float64

In [13]:
len(video_df_merge)

69819

In [ ]:
for i in range(35):
  chat_df = get_chat(key_series = video_df_merge['url'],
                   n = 2000,  # Just the first 100 videos. We'll need this for all the sample.
                   n_message_limit = 20000) ## max retries? Do we need to take those out manually? 30000 is safe tho.
  chat_df.to_csv('chat_df%s.csv' %i)

0.0 https://www.twitch.tv/videos/1663226548 8415
0.0005 https://www.twitch.tv/videos/1663226212 7775
0.001 https://www.twitch.tv/videos/1663206760 6117
0.0015 https://www.twitch.tv/videos/1663191849 8680
0.002 https://www.twitch.tv/videos/1663191728 4888
0.0025 https://www.twitch.tv/videos/1659966453 1563
0.003 https://www.twitch.tv/videos/1659203678 11600
0.0035 https://www.twitch.tv/videos/1659201786 129
0.004 https://www.twitch.tv/videos/1652463497 707
0.0045 https://www.twitch.tv/videos/1652462882 20000
0.005 https://www.twitch.tv/videos/1652462706 3427
0.0055 https://www.twitch.tv/videos/1652462377 1850
0.006 https://www.twitch.tv/videos/1652460934 517
0.0065 https://www.twitch.tv/videos/1652460674 2293
0.007 https://www.twitch.tv/videos/1652460003 2160
0.0075 https://www.twitch.tv/videos/1652459721 469
0.008 https://www.twitch.tv/videos/1648779136 823
0.0085 https://www.twitch.tv/videos/1648024014 805
0.009 https://www.twitch.tv/videos/1648023796 1602
0.0095 https://www.twitch.tv

In [ ]:
print(chat_df.shape)
print(chat_df.columns)
chat_df.head(5)

In [ ]:
chat_df.to_csv('chat_df3.csv')

In [ ]:
# Well, you can even click on the url. That's neat.
chat_df.groupby('video_url')['messages'].count().describe()

count       91.000000
mean      2485.791209
std       4476.312375
min          1.000000
25%        135.500000
50%        988.000000
75%       1970.500000
max      20000.000000
Name: messages, dtype: float64

# **Ignore the code down below**

In [ ]:
## First we take a look at what data there is about the chats
url = video_df['url'][1]
chat = ChatDownloader().get_chat(url)       # create a generator
         # print the formatted message

# For each chat message, this is the full structure
# next(chat)['author']['badges']

pd.DataFrame.from_dict(json_normalize(next(chat)), orient='columns')

,message_id,timestamp,time_in_seconds,source,state,message,time_text,message_type,author.display_name,author.id,author.name,author.type,author.bio,author.created_at,author.images,author.badges,author.colour
0,56dfa958-be91-4bd7-8679-979e1531f545,1665268893255000,10.555,chat,published,We back,0:10,text_message,bobo_n,566592468,bobo_n,user,I would like to remain unknown.Don't check whi...,1597320910088820,[{'url': 'https://static-cdn.jtvnw.net/jtv_use...,"[{'name': 'subscriber', 'version': 12, 'title'...",#FF7F50


In [ ]:
from itertools import chain
temp = pd.io.json.json_normalize(chain.from_iterable(chat))

!pip install flatten_json
from flatten_json import flatten

temp = (flatten(record) for record in top)
temp_df = pd.DataFrame(temp)

temp_df[['message_id', 'timestamp', 'time_in_seconds', 'author_display_name',
       'author_id', 'author_name', 'author_bio','author_created_at','message']]

In [ ]:
# define function to get chat files.
import itertools

def get_chat(key_series = video_df['url'],n = 5, n_message_limit = 100):
  '''
    key_series is a pandas series of video urls.
    n is the number of videos you wanna pull chat from
    n_message_limit is the limit of messages you wanna pull from each video
    '''
  temp_df = pd.DataFrame()
  for i in range(0,n) :
    url = key_series[i]
    chat = ChatDownloader().get_chat(url)   
        
    top = itertools.islice(chat, n_message_limit)
    
    for message in top:     # iterate over messages
      temp_df_temp = pd.DataFrame.from_dict(json_normalize(next(chat)), orient='columns')

      frames = [temp_df, temp_df_temp]
      temp_df = pd.concat(frames, ignore_index=True)
  
  return temp_df

In [ ]:
chat_df = get_chat(key_series = video_df['url'],n = 2, n_message_limit = 5000)

print(chat_df.shape)
print(chat_df.columns)
chat_df.head(5)

(6823, 18)


,message_id,timestamp,time_in_seconds,source,state,message,time_text,message_type,author.display_name,author.id,author.name,author.type,author.bio,author.created_at,author.images,author.badges,author.colour,emotes
0,0067de3a-3127-48e6-b2e8-706a67cfc81e,1665353970195000,36.495,chat,published,Have good steam!,0:36,text_message,Azkii,39378978,azkii,user,"Hello, I am Azkii. Proud member of the Toad Sq...",1358318766728854,[{'url': 'https://static-cdn.jtvnw.net/jtv_use...,"[{'name': 'premium', 'version': 1, 'title': 'P...",#008000,NaN
1,7fdd1f1d-846b-4999-ae1a-d147a4e48503,1665353978426000,44.726,chat,published,yoooo,0:44,text_message,youngmoneybandz,519136869,youngmoneybandz,user,hey nerd,1587643311316222,[{'url': 'https://static-cdn.jtvnw.net/jtv_use...,"[{'name': 'subscriber', 'version': 6, 'title':...",#00FF7F,NaN
2,741657a0-7d3d-4421-adc5-b113a8239428,1665353978988000,45.288,chat,published,shanenobi - iiTzCrosshair list for Valorant ii...,0:45,text_message,StreamElements,100135110,streamelements,user,Elevate your content with www.StreamElements.c...,1440188714224473,[{'url': 'https://static-cdn.jtvnw.net/jtv_use...,"[{'name': 'moderator', 'version': 1, 'title': ...",#5B99FF,[{'id': 'emotesv2_f1794ecf1c0345c98655087a730f...
3,b774da23-2eb2-4a0d-8408-f396d29e3c8a,1665353982651000,48.951,chat,published,yooooooooooooooo,0:48,text_message,acechopper13,505849553,acechopper13,user,I'm just going to do gaming and nothing but ga...,1585541399427960,[{'url': 'https://static-cdn.jtvnw.net/user-de...,"[{'name': 'premium', 'version': 1, 'title': 'P...",#2E8B57,NaN
4,7631c35a-da75-4a16-a668-919cd6876181,1665353984845000,51.145,chat,published,hi Tim,0:51,text_message,Zed_Hennydog,428976858,zed_hennydog,user,Together we thunder 🇿🇲🇳🇿🌍,1554805481749155,[{'url': 'https://static-cdn.jtvnw.net/jtv_use...,"[{'name': 'premium', 'version': 1, 'title': 'P...",#0000FF,NaN


In [ ]:
# This is a demo of one streamer, 5 messages. 
## Now what do we want to put in a dataframe? 
url = video_df['url'][1]
chat = ChatDownloader().get_chat(url)   

timestamp = []
time_in_seconds = []
messages = []
author_display_name = []
# author_id = []
## Let's hold off the badges and emotes. Those are multi-levels. But they'll be super helpful!!

import itertools

top5 = itertools.islice(chat, 5)

for message in top5:                        # iterate over messages    
    timestamp.append(message['timestamp'])
    time_in_seconds.append(message['time_in_seconds'])
    messages.append(message['message'])
    author_display_name.append(message['author']['display_name'])
    ## better way to flatten. 
    ## can be zipped to a function

#for message in chat:                        # iterate over messages. It's not json!! It's just a dict
#    print(json.dumps(message, indent = 4, sort_keys=True))

chat_df = pd.DataFrame(list(zip(timestamp,time_in_seconds,messages,author_display_name)),
                       columns = ['timestamp','time_in_seconds','messages','author_display_name'])

chat_df.head()
## There're emotes! 

,timestamp,time_in_seconds,messages,author_display_name
0,1665286892922000,14.222,chin shane sort dla piscine,stephanokc
1,1665286915688000,36.988,bye la bonne soireee jvous aiment,NurseScraby
2,1665286956005000,77.305,eille salut les boys !,stephanokc
3,1665286976686000,97.986,Salut la! Jva prob encore cornifler demain si ...,ReactHD9
4,1665286982791000,104.091,oublie pas de temercier l'academie,stephanokc


In [ ]:
chat_df = get_chat(key_series = video_df['url'],n = 5, n_message_limit = 100000)

print(chat_df.shape)
chat_df.head(10)

(1198, 5)


,video_url,timestamp,time_in_seconds,messages,author_display_name
0,https://www.twitch.tv/videos/1619335587,1665298566706000,293.006,fergyf1HYPE fergyf1HYPE fergyf1HYPE fergyf1HYPE,FergyFergTV
1,https://www.twitch.tv/videos/1619335587,1665298566890000,293.190,fergyf1HYPE fergyf1HYPE fergyf1HYPE fergyf1HYPE,FergyFergTV
2,https://www.twitch.tv/videos/1619335587,1665298567013000,293.313,fergyf1HYPE fergyf1HYPE fergyf1HYPE fergyf1HYPE,FergyFergTV
3,https://www.twitch.tv/videos/1619335587,1665298816474000,542.774,"What can I serve you? ""!drinks"" to order",L3_Bartender
4,https://www.twitch.tv/videos/1619335587,1665298975363000,701.663,allo!,leGlitch
5,https://www.twitch.tv/videos/1619335587,1665299000653000,726.953,pas pire j't'encore réveillé mais pas pour lon...,leGlitch
6,https://www.twitch.tv/videos/1619335587,1665299006146000,732.446,aaaaahhhh poutine,leGlitch
7,https://www.twitch.tv/videos/1619335587,1665299128101000,854.401,jsuis bleu une débarque en bécique a beu ding ...,leGlitch
8,https://www.twitch.tv/videos/1619335587,1665299165171000,891.471,juste pour la toune,leGlitch
9,https://www.twitch.tv/videos/1619335587,1665299257301000,983.601,je game pas avec mes cuisses donc drille mes c...,leGlitch


In [ ]:
# Well, you can even click on the url. That's neat.
chat_df.groupby('video_url')['messages'].count()

video_url
https://www.twitch.tv/videos/1618617417      5
https://www.twitch.tv/videos/1618629121    144
https://www.twitch.tv/videos/1618770779    366
https://www.twitch.tv/videos/1618936117    224
https://www.twitch.tv/videos/1619238405     10
https://www.twitch.tv/videos/1619335587    449
Name: messages, dtype: int64

Now, we need to extract variables from the chat, merge to video_df.

In [ ]:
 = pd.merge(video_df, temp,  how='left', left_on=['video_url'], right_on = ['video_url'])

Parse

In [ ]:
# No need for this anymore.
## parse chat file
def get_chat_dataframe(file):
    data = []
    with open(file, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
        for line in lines:
            #print(line)
            
            time_logged = line.split('UTC')[0][1:].strip()
            #time_logged = datetime.strptime(time_logged,'%Y-%m-%d-%H:%M:%S')
            #print(time_logged)
            
            username_message = line.split(']')[1:]
            username_message = '—'.join(username_message).strip()
            #print(username_message)
            try:
                temp = username_message.split(":")
                username = temp[0]
                message = temp[1]
            except IndexError:
                username = temp[0]
                message = '-'
            #print(username)
            #print(message)
            
            d = {'dt': time_logged,
                 'username': username,
                 'message': message
                    }
            data.append(d)
            
    return pd.DataFrame().from_records(data)

In [ ]:
## This is an example
chat_df = get_chat_dataframe('[1-21-22] Mongraal - Winning Duo Cash Cup With FaZe BL GUILD Flikk (Webcam Mic) - Chat.txt')

chat_df.head()

Cleaning and preprocessing

In [ ]:
# Custom STOPWORDS removal -- Need to think about this.Could use the cloud as below 
from texthero import stopwords
default_stopwords = stopwords.DEFAULT
custom_stopwords = default_stopwords.union(set(['yo','play',"twitch","make", "use", "thank", "content", "good", "use",
                                                "think", "need", "harrisheller", "like", "stream",
                                               "kekw","catjam","tim","timthetatman","tatkevinh",
                                               "wipz","docspin","pog","tatlove","lol","lul","omegalul",
                                               "biblethump","clap","tathypers","pepeja","kappa","tattopd","ppsmoke",
                                               "pepelaugh","gopackgo","gachihyper","tatkevinh", "wipz",
                                               "pausechamp","yep","lmao","jack","lulw","monkaw","kreygasm",
                                               "pepega","peped","foxsalt","pogchamp","xqcn","get","back",
                                               "tattuff","tatfat","tatpumpkin","lmao","sadge","sippy",
                                               "pogu","poggers","consolecd","widepeepohappy","pogu","tategg2",
                                               "modcheck","timmy","tathmm","tats","got","com","babyrage",
                                               "xqcp","tatw","pokiw","know", "thats","pepocd","tatafk",
                                               "4weird","tatkkevin", "tatblanket","tatglam","tategg1","wutface",
                                               "blobdance", "kapp","tatbruh","kappapride","facebaby","xqc",
                                               "xqcm","bora","hyperclap","tatlit","5head","gachibass", "go", "ur",
                                                "yes","going","would","im","oh","dez","taty","tk","u","sg", "dont",
                                                "hey","hf","look","anita","anitaheart","leeper","anitahey",
                                                "anitadab","anitadoubt","anitahands", "anitalul","anitapog","anitasword",
                                                "anitaprime","anitafeels","residentsleeper","anitapride","anital",
                                                "anitabiscuit", "thing","widepeeposad","anitahype","among", "lesbianpride",
                                                "heyguys","seemsgood","notlikethis","hi","also","much","yeah",
                                               ])) ## Add as per requirement
# data = hero.remove_stopwords(data, default_stopwords)
chat_df_clean = hero.remove_stopwords(chat_df_temp, custom_stopwords)

In [56]:
# define function to get chat files.
import itertools

def get_chat(key_series = video_df_merge['url'],n = 5, n_message_limit = 1000):
  '''
    key_series is a pandas series of video urls.
    n is the number of videos you wanna pull chat from
    n_message_limit is the limit of messages you wanna pull from each video
    '''
  df = pd.DataFrame(columns=['message_id', 'timestamp', 'time_in_seconds', 'author_display_name',
       'author_id', 'author_name', 'author_bio','author_created_at','message'])

  for i in range(0,n+1) :
    url = key_series[i]
    chat = ChatDownloader().get_chat(url)    
    top = itertools.islice(chat, n_message_limit)
    
    counter = sum(1 for dummy in top)

    temp = (flatten(record) for record in top)
    temp_df = pd.DataFrame(temp)
    temp_df = temp_df[['message_id', 'timestamp', 'time_in_seconds', 'author_display_name',
       'author_id', 'author_name', 'author_bio','author_created_at','message']]
    
    df.append(temp_df)
      
    print(i / n, url,counter)
  
  return chat_df